In [1]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta, timezone

# Binance 연결
exchange = ccxt.binance()

# 심볼 리스트
symbols = [
    'BTC/USDT','ETH/USDT'
]

# 오늘 날짜 (UTC 기준)
today = datetime.utcnow().replace(tzinfo=timezone.utc)

# 끝 시점: 이틀 전 00:00
end_time = datetime(today.year, today.month, today.day, tzinfo=timezone.utc) - timedelta(days=2)

# 시작 시점: 끝 시점으로부터 7일 전
start_time = end_time - timedelta(days=7)

since = int(start_time.timestamp() * 1000)
until = int(end_time.timestamp() * 1000)

print("데이터 수집 구간:", start_time, " ~ ", end_time)

data = {}
for symbol in symbols[:]:  # 예시로 5개만
    all_ohlcv = []
    fetch_since = since
    
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1m', since=fetch_since, limit=1000)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        
        last_ts = ohlcv[-1][0]
        if last_ts >= until:
            break
        fetch_since = last_ts + 60_000  # 1분 = 60,000ms
    
    df = pd.DataFrame(all_ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    df['symbol'] = symbol 
    df.set_index('datetime', inplace=True)
    
    df = df.loc[start_time:end_time]
    data[symbol] = df


data = pd.concat(data.values())

C:\Users\User\AppData\Local\Temp\ipykernel_22228\934743543.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().replace(tzinfo=timezone.utc)


데이터 수집 구간: 2025-09-06 00:00:00+00:00  ~  2025-09-13 00:00:00+00:00


In [2]:
data['symbol'].nunique()

2